In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

CIFAR 100 dataset has 20 superclasses each containing 5 subclasses.

So now I will be training 20 different models on each of the superclasses and then train the gating model for Mixture Of Experts.

So now for the 20 different models, I will be creating seperate training data and labels.

In [2]:
#This dictionary maps 100 fine grained classes to 20 coearse grained superclasses.

fine_to_coarse_labels = {
    0: 4, 1: 1, 2: 14, 3: 8, 4: 0, 5: 6, 6: 7, 7: 7, 8: 18, 9: 3, 10: 3,
    11: 14, 12: 9, 13: 18, 14: 7, 15: 11, 16: 3, 17: 9, 18: 11, 19: 13,
    20: 16, 21: 17, 22: 12, 23: 15, 24: 17, 25: 12, 26: 18, 27: 1, 28: 19,
    29: 8, 30: 10, 31: 6, 32: 5, 33: 1, 34: 15, 35: 18, 36: 16, 37: 10,
    38: 19, 39: 14, 40: 1, 41: 5, 42: 19, 43: 1, 44: 1, 45: 15, 46: 12,
    47: 17, 48: 18, 49: 12, 50: 10, 51: 4, 52: 18, 53: 17, 54: 10, 55: 16,
    56: 19, 57: 2, 58: 17, 59: 16, 60: 4, 61: 10, 62: 2, 63: 11, 64: 11,
    65: 9, 66: 16, 67: 11, 68: 6, 69: 11, 70: 3, 71: 4, 72: 15, 73: 13,
    74: 9, 75: 9, 76: 8, 77: 17, 78: 10, 79: 7, 80: 1, 81: 12, 82: 15,
    83: 13, 84: 2, 85: 2, 86: 1, 87: 12, 88: 11, 89: 3, 90: 2, 91: 13,
    92: 14, 93: 16, 94: 8, 95: 19, 96: 14, 97: 1, 98: 16, 99: 8
}

superclass_names = [
    "aquatic mammals", "fish", "flowers", "food containers", "fruit and vegetables",
    "household electrical devices", "household furniture", "insects", "large carnivores",
    "large man-made outdoor things", "large natural outdoor scenes",
    "large omnivores and herbivores", "medium-sized mammals", "non-insect invertebrates",
    "people", "reptiles", "small mammals", "trees", "vehicles 1", "vehicles 2"
]

def get_superclass_data():
    (x_train_fine, y_train_fine), (x_test_fine, y_test_fine) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

    superclass_data = {}

    for name in superclass_names:
        superclass_data[name] = {'images':[], 'labels':[]}

    for i in range(len(x_train_fine)):
        fine_label = y_train_fine[i][0]
        coarse_label = fine_to_coarse_labels[fine_label]
        superclass_name = superclass_names[coarse_label]

        superclass_data[superclass_name]['images'].append(x_train_fine[i])
        superclass_data[superclass_name]['labels'].append(fine_label)

    for name in superclass_names:
        images = np.array(superclass_data[name]['images'])
        labels = np.array(superclass_data[name]['labels'])
        superclass_data[name] = (images,labels)
    
    return superclass_data

In [3]:
superclass_data = get_superclass_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


In [7]:
all_data = list(superclass_data.values())

In [11]:
all_images = []
all_labels = []

for tup in all_data:
    tup_l = list(tup)
    all_images.append(tup_l[0])
    all_labels.append(tup_l[1])
    

Now building and training 20 different models each for a different superclass.

In [26]:
def build_model(num_classes):
    model = Sequential([
       # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        # Flatten the feature maps to feed into the dense layers
        Flatten(),
        # Fully Connected Layers
        Dense(128, activation='relu'),
        # Output layer with 'num_classes' neurons for classification
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

    return model

def train_single_superclass_model(x_train_superclass,y_train_superclass,superclass_name):
    unique_label = np.unique(y_train_superclass)
    num_classes = len(unique_labels)
    label_map = {label: i for i, label in enumerate(unique_labels)}
    y_train_mapped = np.array([label_map[label] for label in y_train_superclass])

    print(f"\n--- Building and training model for superclass: {superclass_name} ---")
    print(f"Number of unique fine classes: {num_classes}")

    model = build_model(num_classes)
    model.fit(x_train_superclass, y_train_mapped, epochs = 5,batch_size = 32, verbose = 1)

    print(f"Training for {superclass_name} complete.")
    return model

In [30]:
aq_m_x_train = all_images[0]
fish_x_train = all_images[1]
flwr_x_train = all_images[2]
fooc_x_train = all_images[3]
fav_x_train = all_images[4]
hed_x_train = all_images[5]
hf_x_train = all_images[6]
ins_x_train = all_images[7]
lc_x_train = all_images[8]
lmot_x_train = all_images[9]
lnos_x_train = all_images[10]
loah_x_train = all_images[11]
msm_x_train = all_images[12]
nii_x_train = all_images[13]
peo_x_train = all_images[14]
rep_x_train = all_images[15]
sm_x_train = all_images[16]
trees_x_train = all_images[17]
v1_x_train = all_images[18]
v2_x_train = all_images[19]


aq_m_y_train = all_labels[0].reshape(-1,1)
fish_y_train = all_labels[1].reshape(-1,1)
flwr_y_train = all_labels[2].reshape(-1,1)
fooc_y_train = all_labels[3].reshape(-1,1)
fav_y_train = all_labels[4].reshape(-1,1)
hed_y_train = all_labels[5].reshape(-1,1)
hf_y_train = all_labels[6].reshape(-1,1)
ins_y_train = all_labels[7].reshape(-1,1)
lc_y_train = all_labels[8].reshape(-1,1)
lmot_y_train = all_labels[9].reshape(-1,1)
lnos_y_train = all_labels[10].reshape(-1,1)
loah_y_train = all_labels[11].reshape(-1,1)
msm_y_train = all_labels[12].reshape(-1,1)
nii_y_train = all_labels[13].reshape(-1,1)
peo_y_train = all_labels[14].reshape(-1,1)
rep_y_train = all_labels[15].reshape(-1,1)
sm_y_train = all_labels[16].reshape(-1,1)
trees_y_train = all_labels[17].reshape(-1,1)
v1_y_train = all_labels[18].reshape(-1,1)
v2_y_train = all_labels[19].reshape(-1,1)